# Lesson 2 - Pydantic Agents with tools

In this lesson you will learn how to build an agent that uses tools to complete tasks given by the user.

Tools are super useful, as they give the agent extra skills.<br/>
They might even help them count Rs in strawberry 🍓

## Create the first tool - `character_counting_tool`

Let's create a tool that can count characters in a given string.
The good news is that Python has a function to do just that.

In [ ]:
count = "Strawberry".lower().count("R".lower())
print(count)

 A good tool is made of two parts:
* **The Code** – that performs the task
* **A Clear Description** – for the agent telling it: what the tool does, how to call it, and what to expect in return.

There are two ways we could provide the tool description with Docstring or Pydantic's annotation.

### Docstring style

In [ ]:
def character_counting_tool(
    text: str,
    character: str
) -> int:
    """TODO-description-goes-here--tell-the-agent-the-purpose-of-this-function
    
    Args:
        text: TODO-text-property-description-here
        character: TODO-character-property-description-here

    Returns:
        TODO-return-response-goes-here
    """

    count = text.lower().count(character.lower())
    return count

### Pydantic's annotations style

In [ ]:
from typing import Annotated
from pydantic import Field

def character_counting_tool(
    text:      Annotated[str, Field(description="TODO-text-property-description-here")],
    character: Annotated[str, Field(description="TODO-character-property-description-here")],
) -> Annotated[int, Field(description="TODO-return-response-goes-here")]:
    """TODO-description-goes-here--tell-the-agent-the-purpose-of-this-function
    """

    print(f"🤖 TOOL: character_counting_tool({text}, {character})")

    count = text.lower().count(character.lower())
    print(count)
    return count

### Refusal and Nickname tools

It is also great to give a way out for our agents in case they don't have the necessary tools to complete the task.<br/>
This is where a refusal tool comes into play.

In the refusal tool tell the agent to:
* use nickname_tool to respond
* appologise
* list tools (maybe tell it to skip refusal_tool and nickname_tool)
* give it a fun hint (like ask my overlord to upgrade me)

Also, let's have some fun, and call the user some nice nicknames.

In [ ]:
def refusal_tool() -> Annotated[str, Field(description="The response provides you with instructions on how to respond to the user")]:
    """This is a refusal tool, it tells you how to respond to the user"""
    print("🤖 TOOL: refusal_tool()")

    return """
        Appologise that you can't help them.
        TODO: add the rest of the error message
        """

In [ ]:
import random
names = ["Buddy", "Friend", "Boss", "Gorgeous"]

def nickname_tool() -> Annotated[str, Field(description="An endearing name, you can use to call the user")]:
    """Returns a random name that can be used to call the user"""
    print("🤖 TOOL: nickname_tool()")


    random_index = random.randint(0,len(names)-1)
    return names[random_index]

## Create an agent with Tools

In [ ]:
from pydantic_ai import Agent
from pydantic_ai.tools import Tool

def prepare_agent() -> Agent:
    return Agent(
        model="bedrock:anthropic.claude-3-5-sonnet-20241022-v2:0",
        instructions="""
        You are a helpful assistant answering any questions a user may have.
        You have access to the following tools:
        - character_counting_tool

        You should only try to provide the response only with the provided tools.
        Use refusal_tool to understand how to respond when you don't have the tools to answer.

        When responding to the user, always use a nickname, you must get the nickname from nickname_tool.
        """,
        tools=[
            Tool(character_counting_tool, takes_ctx=False),
            # TODO: add refusal_tool
            # TODO: add nickname_tool
        ]
    )

agent = prepare_agent()

In [ ]:
response = await agent.run("What is the fastest way to Mars?")
print(response.output)

In [ ]:
response = await agent.run("How many Rs are there in the Polish word truskawka when translated to English?")
print(response.output)

## Have fun